<picture>
  <source media="(prefers-color-scheme: dark)" srcset="https://vespa.ai/assets/vespa-ai-logo-heather.svg">
  <source media="(prefers-color-scheme: light)" srcset="https://vespa.ai/assets/vespa-ai-logo-rock.svg">
  <img alt="#Vespa" width="200" src="https://vespa.ai/assets/vespa-ai-logo-rock.svg" style="margin-bottom: 25px;">
</picture>

# Feeding performance

This explorative notebook intends to shine some light on the different modes of feeding documents to Vespa.
We will look at these different methods:

1. Sync feeding from pyVespa.
2. Async feeding from pyVespa.
3. Using `app.feed_iterable()` from pyVespa.
4. Using Vespa CLI[TODO:Link].


<div class="alert alert-info">
    Refer to <a href="https://pyvespa.readthedocs.io/en/latest/troubleshooting.html">troubleshooting</a>
    for any problem when running this guide.
</div>


[Install pyvespa](https://pyvespa.readthedocs.io/) and start Docker Daemon, validate minimum 6G available:


In [1]:
#!pip3 install pyvespa
#!docker info | grep "Total Memory"

## Create an application package

The [application package](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.package.ApplicationPackage)
has all the Vespa configuration files -
create one from scratch:


In [1]:
from vespa.package import (
    ApplicationPackage,
    Field,
    Schema,
    Document,
    FieldSet,
)

package = ApplicationPackage(
    name="pyvespafeed",
    schema=[
        Schema(
            name="doc",
            document=Document(
                fields=[
                    Field(name="id", type="string", indexing=["summary"]),
                    Field(name="text", type="string", indexing=["summary"]),
                ]
            ),
            fieldsets=[FieldSet(name="default", fields=["text"])],
        )
    ],
)

Note that the name cannot have `-` or `_`.


## Deploy the Vespa application

Deploy `package` on the local machine using Docker,
without leaving the notebook, by creating an instance of
[VespaDocker](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.deployment.VespaDocker). `VespaDocker` connects
to the local Docker daemon socket and starts the [Vespa docker image](https://hub.docker.com/r/vespaengine/vespa/).

If this step fails, please check
that the Docker daemon is running, and that the Docker daemon socket can be used by clients (Configurable under advanced settings in Docker Desktop).


In [8]:
from vespa.deployment import VespaDocker

vespa_docker = VespaDocker()
app = vespa_docker.deploy(application_package=package)

Waiting for configuration server, 0/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 0/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 5/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 10/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Application is up!
Finished deployment.


`app` now holds a reference to a [Vespa](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.application.Vespa) instance.


## Feeding documents to Vespa

In this example we use the [HF Datasets](https://huggingface.co/docs/datasets/index) library to stream the
[BeIR/nfcorpus](https://huggingface.co/datasets/BeIR/nfcorpus) dataset and index in our newly deployed Vespa instance. Read
more about the [NFCorpus](https://www.cl.uni-heidelberg.de/statnlpgroup/nfcorpus/):

> NFCorpus is a full-text English retrieval data set for Medical Information Retrieval.

The following uses the [stream](https://huggingface.co/docs/datasets/stream) option of datasets to stream the data without
downloading all the contents locally. The `map` functionality allows us to convert the
dataset fields into the expected feed format for `pyvespa` which expects a dict with the keys `id` and `fields`:

`{ "id": "vespa-document-id", "fields": {"vespa_field": "vespa-field-value"}}`


In [ ]:
from datasets import load_dataset

lang = "simple"  # Use the Simple English Wikipedia subset
docs = load_dataset(
    "Cohere/wikipedia-2023-11-embed-multilingual-v3",
    lang,
    split="train",
    streaming=True,
)
for doc in docs:
    doc_id = doc["_id"]
    title = doc["title"]
    text = doc["text"]
    emb = doc["emb"]

In [13]:
from datasets import load_dataset

dataset = load_dataset(
    "Cohere/wikipedia-2023-11-embed-multilingual-v3",
    "ext",
    split="train",
    streaming=False,
)
vespa_feed = dataset.map(lambda x: {"id": x["_id"], "fields": {"text": x["text"]}})

Generating train split: 10967 examples [00:00, 113213.37 examples/s]
Map: 100%|██████████| 10967/10967 [00:00<00:00, 33289.33 examples/s]


Now we can feed to Vespa using `feed_iterable` which accepts any `Iterable` and an optional callback function where we can
check the outcome of each operation. The application is configured to use [embedding](https://docs.vespa.ai/en/embedding.html)
functionality, that produce a vector embedding using a concatenation of the title and the body input fields. This step is computionally expensive. Read more
about embedding inference in Vespa in the [Accelerating Transformer-based Embedding Retrieval with Vespa](https://blog.vespa.ai/accelerating-transformer-based-embedding-retrieval-with-vespa/).


In [8]:
"""
A rudimentary URL downloader (like wget or curl) to demonstrate Rich progress bars.
"""

import os.path
import sys
from concurrent.futures import ThreadPoolExecutor
import signal
from functools import partial
from threading import Event
from typing import Iterable
from urllib.request import urlopen

from rich.progress import (
    BarColumn,
    DownloadColumn,
    Progress,
    TaskID,
    TextColumn,
    TimeRemainingColumn,
    TransferSpeedColumn,
)

progress = Progress(
    TextColumn("[bold blue]{task.fields[filename]}", justify="right"),
    BarColumn(bar_width=None),
    "[progress.percentage]{task.percentage:>3.1f}%",
    "•",
    DownloadColumn(),
    "•",
    TransferSpeedColumn(),
    "•",
    TimeRemainingColumn(),
)


done_event = Event()


def handle_sigint(signum, frame):
    done_event.set()


signal.signal(signal.SIGINT, handle_sigint)


def copy_url(task_id: TaskID, url: str, path: str) -> None:
    """Copy data from a url to a local file."""
    progress.console.log(f"Requesting {url}")
    response = urlopen(url)
    # This will break if the response doesn't contain content length
    progress.update(task_id, total=int(response.info()["Content-length"]))
    with open(path, "wb") as dest_file:
        progress.start_task(task_id)
        for data in iter(partial(response.read, 32768), b""):
            dest_file.write(data)
            progress.update(task_id, advance=len(data))
            if done_event.is_set():
                return
    progress.console.log(f"Downloaded {path}")


def download(urls: Iterable[str], dest_dir: str):
    """Download multiple files to the given directory."""

    with progress:
        with ThreadPoolExecutor(max_workers=4) as pool:
            for url in urls:
                filename = url.split("/")[-1]
                dest_path = os.path.join(dest_dir, filename)
                task_id = progress.add_task("download", filename=filename, start=False)
                pool.submit(copy_url, task_id, url, dest_path)


download(
    [
        "www.wikipedia.org",
    ]
    * 10,
    ".",
)

Output()

[18:54:24] Requesting www.wikipedia.org                                                            ]8;id=256028;file:///var/folders/vb/ch14y_kn4mqfz75bhc9_g5980000gn/T/ipykernel_5193/3706060488.py\3706060488.py]8;;\:]8;id=272410;file:///var/folders/vb/ch14y_kn4mqfz75bhc9_g5980000gn/T/ipykernel_5193/3706060488.py#49\49]8;;\

           Requesting www.wikipedia.org                                                            ]8;id=761615;file:///var/folders/vb/ch14y_kn4mqfz75bhc9_g5980000gn/T/ipykernel_5193/3706060488.py\3706060488.py]8;;\:]8;id=982730;file:///var/folders/vb/ch14y_kn4mqfz75bhc9_g5980000gn/T/ipykernel_5193/3706060488.py#49\49]8;;\

           Requesting www.wikipedia.org                                                            ]8;id=717322;file:///var/folders/vb/ch14y_kn4mqfz75bhc9_g5980000gn/T/ipykernel_5193/3706060488.py\3706060488.py]8;;\:]8;id=542799;file:///var/folders/vb/ch14y_kn4mqfz75bhc9_g5980000gn/T/ipykernel_5193/3706060488.py#49\49]8;;\

           Requesting www.wikipedia.org                                                            ]8;id=445750;file:///var/folders/vb/ch14y_kn4mqfz75bhc9_g5980000gn/T/ipykernel_5193/3706060488.py\3706060488.py]8;;\:]8;id=268876;file:///var/folders/vb/ch14y_kn4mqfz75bhc9_g5980000gn/T/ipykernel_5193/3706060488.py#49\49]8;;\

           Requesting www.wikipedia.org                                                            ]8;id=602741;file:///var/folders/vb/ch14y_kn4mqfz75bhc9_g5980000gn/T/ipykernel_5193/3706060488.py\3706060488.py]8;;\:]8;id=747599;file:///var/folders/vb/ch14y_kn4mqfz75bhc9_g5980000gn/T/ipykernel_5193/3706060488.py#49\49]8;;\

In [1]:
from tqdm.rich import trange, tqdm
import time

for i in trange(0, 100):
    time.sleep(0.1)

Output()

/Users/thomas/Repos/pyvespa/pyvespa/lib/python3.9/site-packages/tqdm/rich.py:145: TqdmExperimentalWarning: rich is experimental/alpha
  return tqdm_rich(range(*args), **kwargs)


In [18]:
vespa_feed.add_elasticsearch_index(index_name="simple", url="http://localhost:8080")

Dataset({
    features: ['_id', 'url', 'title', 'text', 'emb', 'id', 'fields'],
    num_rows: 10967
})

In [17]:
from vespa.io import VespaResponse, VespaQueryResponse


def callback(response: VespaResponse, id: str):
    print(response)
    if not response.is_successful():
        print(f"Error when feeding document {id}: {response.get_json()}")


app.feed_iterable(
    vespa_feed[:1], schema="doc", namespace="pyvespa-feed", callback=callback
)

Exception in thread Thread-9:
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/thomas/Repos/pyvespa/pyvespa/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/thomas/Repos/pyvespa/vespa/application.py", line 365, in _consumer
    _handle_result_callback(future, callback=callback)
  File "/Users/thomas/Repos/pyvespa/vespa/application.py", line 418, in _handle_result_callback
    id, response = future.result()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/concurrent/futures/_base.py", line 438, in result
  

KeyboardInterrupt: 

## Querying Vespa

Using the [Vespa Query language](https://docs.vespa.ai/en/query-language.html) we can query the indexed data.

- Using a context manager `with app.syncio() as session` to handle connection pooling ([best practices](https://cloud.vespa.ai/en/http-best-practices))
- The query method accepts any valid Vespa [query api parameter](https://docs.vespa.ai/en/reference/query-api-reference.html) in `**kwargs`
- Vespa api parameter names that contains `.` must be sent as `dict` parameters in the `body` method argument

The following searches for `How Fruits and Vegetables Can Treat Asthma?` using different retrieval and [ranking](https://docs.vespa.ai/en/ranking.html) strategies.


In [5]:
import pandas as pd


def display_hits_as_df(response: VespaQueryResponse, fields) -> pd.DataFrame:
    records = []
    for hit in response.hits:
        record = {}
        for field in fields:
            record[field] = hit["fields"][field]
        records.append(record)
    return pd.DataFrame(records)

### Plain Keyword search

The following uses plain keyword search functionality with [bm25](https://docs.vespa.ai/en/reference/bm25.html) ranking, the `bm25` rank-profile was configured in the
application package to use a linear combination of the bm25 score of the query terms against the title and the body field.


In [6]:
with app.syncio(connections=1) as session:
    query = "How Fruits and Vegetables Can Treat Asthma?"
    response: VespaQueryResponse = session.query(
        yql="select * from sources * where userQuery() limit 5",
        query=query,
        ranking="bm25",
    )
    assert response.is_successful()
    print(display_hits_as_df(response, ["id", "title"]))

         id                                              title
0  MED-2450  Protective effect of fruits, vegetables and th...
1  MED-2464  Low vegetable intake is associated with allerg...
2  MED-1162  Pesticide residues in imported, organic, and "...
3  MED-2461  The association of diet with respiratory sympt...
4  MED-2458  Manipulating antioxidant intake in asthma: a r...


### Plain Semantic Search

The following uses dense vector representations of the query and the document and matching is performed and accelerated by Vespa's support for
[approximate nearest neighbor search](https://docs.vespa.ai/en/approximate-nn-hnsw.html).
The vector embedding representation of the text is obtained using Vespa's [embedder functionality](https://docs.vespa.ai/en/embedding.html#embedding-a-query-text).


In [7]:
with app.syncio(connections=1) as session:
    query = "How Fruits and Vegetables Can Treat Asthma?"
    response: VespaQueryResponse = session.query(
        yql="select * from sources * where ({targetHits:1000}nearestNeighbor(embedding,q)) limit 5",
        query=query,
        ranking="semantic",
        body={"input.query(q)": f"embed({query})"},
    )
    assert response.is_successful()
    print(display_hits_as_df(response, ["id", "title"]))

         id                                              title
0  MED-5072  Lycopene-rich treatments modify noneosinophili...
1  MED-2472  Vegan regimen with reduced medication in the t...
2  MED-2464  Low vegetable intake is associated with allerg...
3  MED-2458  Manipulating antioxidant intake in asthma: a r...
4  MED-2450  Protective effect of fruits, vegetables and th...


### Hybrid Search

This is one approach to combine the two retrieval strategies and where we use Vespa's support for
[cross-hits feature normalization and reciprocal rank fusion](https://docs.vespa.ai/en/phased-ranking.html#cross-hit-normalization-including-reciprocal-rank-fusion). This
functionality is exposed in the context of `global` re-ranking, after the distributed query retrieval execution which might span 1000s of nodes.

#### Hybrid search with the OR query operator

This combines the two methods using logical disjunction (OR). Note that the first-phase expression in our `fusion` expression is only using the semantic score, this
because usually semantic search provides better recall than sparse keyword search alone.


In [8]:
with app.syncio(connections=1) as session:
    query = "How Fruits and Vegetables Can Treat Asthma?"
    response: VespaQueryResponse = session.query(
        yql="select * from sources * where userQuery() or ({targetHits:1000}nearestNeighbor(embedding,q)) limit 5",
        query=query,
        ranking="fusion",
        body={"input.query(q)": f"embed({query})"},
    )
    assert response.is_successful()
    print(display_hits_as_df(response, ["id", "title"]))

         id                                              title
0  MED-2464  Low vegetable intake is associated with allerg...
1  MED-2450  Protective effect of fruits, vegetables and th...
2  MED-2458  Manipulating antioxidant intake in asthma: a r...
3  MED-2461  The association of diet with respiratory sympt...
4  MED-5069  From beans to berries and beyond: teamwork bet...


#### Hybrid search with the RANK query operator


This combines the two methods using the [rank](https://docs.vespa.ai/en/reference/query-language-reference.html#rank) query operator. In this case
we express that we want to retrieve the top-1000 documents using vector search, and then have sparse features like BM25 calculated as well (second operand
of the rank operator). Finally the hits are re-ranked using the reciprocal rank fusion


In [9]:
with app.syncio(connections=1) as session:
    query = "How Fruits and Vegetables Can Treat Asthma?"
    response: VespaQueryResponse = session.query(
        yql="select * from sources * where rank({targetHits:1000}nearestNeighbor(embedding,q), userQuery()) limit 5",
        query=query,
        ranking="fusion",
        body={"input.query(q)": f"embed({query})"},
    )
    assert response.is_successful()
    print(display_hits_as_df(response, ["id", "title"]))

         id                                              title
0  MED-2464  Low vegetable intake is associated with allerg...
1  MED-2450  Protective effect of fruits, vegetables and th...
2  MED-2458  Manipulating antioxidant intake in asthma: a r...
3  MED-2461  The association of diet with respiratory sympt...
4  MED-5069  From beans to berries and beyond: teamwork bet...


#### Hybrid search with filters

In this example we add another query term to the yql, restricting the nearest neighbor search to only consider documents that have vegetable in the title.


In [10]:
with app.syncio(connections=1) as session:
    query = "How Fruits and Vegetables Can Treat Asthma?"
    response: VespaQueryResponse = session.query(
        yql='select * from sources * where title contains "vegetable" and rank({targetHits:1000}nearestNeighbor(embedding,q), userQuery()) limit 5',
        query=query,
        ranking="fusion",
        body={"input.query(q)": f"embed({query})"},
    )
    assert response.is_successful()
    print(display_hits_as_df(response, ["id", "title"]))

         id                                              title
0  MED-2464  Low vegetable intake is associated with allerg...
1  MED-2450  Protective effect of fruits, vegetables and th...
2  MED-3199  Potential risks resulting from fruit/vegetable...
3  MED-2085  Antiplatelet, anticoagulant, and fibrinolytic ...
4  MED-4496  The effect of fruit and vegetable intake on ri...


## Cleanup


In [6]:
vespa_docker.container.stop()
vespa_docker.container.remove()

## Next steps

This is just an intro into the capabilities of Vespa and pyvespa.
Browse the site to learn more about schemas, feeding and queries -
find more complex applications in
[examples](https://pyvespa.readthedocs.io/en/latest/examples.html).
